# Summarization Using LLM Wrapper

In [1]:
import sys
current_dir = "/Users/josephtolsma/Documents/dev/yelp_rag"
sys.path.insert(0,current_dir)

In [2]:
import pandas as pd
import os
import requests
import src.config as config

In [3]:
pd.set_option("display.max_colwidth",40)

In [4]:
relevant_chunk_df = pd.read_parquet(os.path.join(config.DATA_DIR_PROC,"topic_relevant_review_chunks.parquet"))

In [5]:
relevant_chunk_df.head(3)

,topic,scores,indicies,chunk_id,business_id,restaurant_name,stars,review_id,chunk
0,food,0.512154,69,nE8A-emvMc-8sJ_0_BWYsg_0,IS0B5nLJPluVT8NwGgospA,Cheddar's Scratch Kitchen,5.0,nE8A-emvMc-8sJ_0_BWYsg,This place is ahhmazzing! Every entr...
1,food,0.506828,95,7FzqzVUwsBW_JEjW94fpYQ_0,IS0B5nLJPluVT8NwGgospA,Cheddar's Scratch Kitchen,4.0,7FzqzVUwsBW_JEjW94fpYQ,"My parents were in town visiting, so..."
2,food,0.498265,78,4SDns1Llu0RKzpKZ7w4iKg_0,IS0B5nLJPluVT8NwGgospA,Cheddar's Scratch Kitchen,3.0,4SDns1Llu0RKzpKZ7w4iKg,"This is your usual standard fare, no..."


In [6]:
pd.set_option("display.max_colwidth",None)

**Build Content for Summarization for One Restaurant**

In [7]:
rid = relevant_chunk_df.sample(random_state=42)["business_id"].values[0]

In [8]:
chunk_test_df = relevant_chunk_df[relevant_chunk_df["business_id"] == rid]

In [9]:
topic_context_dict = {}

for topic in config.TOPICS.keys():
    chunk_topic_df = chunk_test_df[chunk_test_df["topic"]==topic]
    context = f"""
    You are analyzing and synthesizing customer reviews for a restaurant called {chunk_test_df["restaurant_name"].values[0]}.

    The following are excerpts from different customer reviews. 
    Each review comes from a different reviewer.

    """
    context+=f"""
    Topic: {topic.upper()}
    """
    i = 0
    for row in chunk_topic_df.iterrows():

        verbatim = f"""
        [{i}] Review ({row[1].stars} stars):
        {row[1].chunk}
        """
        context+=verbatim
        i+=1
    topic_context_dict.update({topic:context})

In [10]:
def call_ollama(
        prompt,
        model,
        temperature=0.2
    ):
    
    url = "http://localhost:11434/api/generate"
    payload = {
        "model":model,
        "prompt":prompt,
        "stream":False,
        "options":{"temperature":temperature}
    }

    resp = requests.post(url,json=payload,timeout=120)
    resp.raise_for_status()
    
    data = resp.json()
    return data["response"].strip()

In [11]:
print(call_ollama(prompt = "say hello.",model = config.OLLAMA_MODEL,temperature=0.2))

Hello! How are you today?


In [12]:
summary_dict = {}
for topic in config.TOPICS.keys():
    chunk_topic_df = chunk_test_df[chunk_test_df["topic"]==topic]
    restaurant_name = chunk_test_df["restaurant_name"].values[0]
    context = f"""
    You are analyzing and synthesizing customer reviews for a restaurant called {restaurant_name}, where the main topic is:
    
    Topic: {topic.upper()}

    Using only the reviews included here, provide a concise (2-3 sentence) summary of {restaurant_name}'s performance in the area of {topic}.

    Guidelines:
    - Capture common themes and disagreements
    - Mention both positives and negatives if present
    - Be factual and grounded in the evidence
    - Do not invent details not supported by the excerpts
    - Do not announce the summary ("here is a concise summary..."); just start with summary content

    """
    i = 0
    for row in chunk_topic_df.iterrows():

        verbatim = f"""
        [{i}] Review ({row[1].stars} stars):
        {row[1].chunk}
        """
        context+=verbatim
        i+=1
    
    summary = call_ollama(prompt = context,
                          model = config.OLLAMA_MODEL,
                          temperature=config.TEMPERATURE
                        )
    summary_dict.update({topic:summary})

In [13]:
pd.DataFrame(summary_dict,index = [0])

,food,service,ambiance
0,"Ants Pants Café's food offerings receive mixed reviews, but overall, customers praise the quality of certain dishes such as sweet potato fries, egg-based entrees (e.g., eggs benedict, omelets), and brunch specials. However, some reviewers express disappointment with other menu items, citing underwhelming flavors or poorly executed dishes like dill-scrambled eggs. The café's ability to execute consistent flavors is questioned, but the quality of ingredients and presentation are generally commended.","Ants Pants Café's service has been inconsistent, with some reviewers experiencing friendly and attentive waitstaff, while others have reported rude, indifferent, or incompetent service. Some common complaints include long wait times, slow service, and a lack of attention to customers' needs. However, a few reviewers have had positive experiences with friendly staff, suggesting that the issue may be more related to staffing levels or training rather than a fundamental problem with the café's culture.","Ants Pants Café's ambiance is generally praised as cozy and welcoming, with reviewers noting the excellent music setting, bright lighting, and ""cute"" or ""low key"" atmosphere. However, some reviewers criticize the high noise levels, particularly when large groups of loud patrons are present, which can detract from the dining experience. The small size of the café is also sometimes seen as a drawback, leading to long wait times and overcrowding during peak hours."


**Build Content for Summarization for All Restaurants**

In [14]:
restaurant_ids = relevant_chunk_df["business_id"].unique().tolist()

In [ ]:
summary_dict = {}
for id in restaurant_ids:
    restaurant_df = relevant_chunk_df[relevant_chunk_df["business_id"] == id]
    restaurant_name = restaurant_df.iloc[0]["restaurant_name"]
    restaurant_summary_dict = {}
    for topic in config.TOPICS.keys():
        topic_df = restaurant_df[restaurant_df["topic"]==topic]
        context = f"""
        You are analyzing and synthesizing customer reviews for a restaurant called {restaurant_name}, where the main topic is:
        
        Topic: {topic.upper()}

        Using only the reviews included here, provide a concise (2-3 sentence) summary of {restaurant_name}'s performance in the area of {topic}.

        Guidelines:
        - Capture common themes, impressions, and disagreements
        - Be factual and grounded in the evidence; do not invent details not supported by the excerpts
        - Begin by characterizing the group of reviews as "positive", "negative", or "mixed" 

        Output to avoid:
        - Content unrelated to the topic (for example: "noise levels" are ambiance, not food)
        - Review indicies (for example, "review 3"); those are not user-facing
        - Announcing the summary ("here is a concise summary..."); just start with summary content
        - Use of the word "cozy"
        - Mention of "hard surfaces"

        """
        i = 0
        for row in chunk_topic_df.iterrows():

            verbatim = f"""
            [{i}] Review ({row[1].stars} stars):
            {row[1].chunk}
            """
            context+=verbatim
            i+=1
        
        summary = call_ollama(prompt = context,
                            model = config.OLLAMA_MODEL,
                            temperature=config.TEMPERATURE
                            )
        restaurant_summary_dict.update({topic:summary})
    summary_dict.update({restaurant_name:restaurant_summary_dict})

In [17]:
pd.DataFrame(summary_dict).T

,food,service,ambiance
Cheddar's Scratch Kitchen,"Mixed performance in the area of food, with some reviewers praising high quality and delicious dishes (e.g. pumpkin pancakes, sweet potato fries, eggs Benedict), while others found the food to be ""unexceptional"" or not memorable. Some reviewers specifically mentioned enjoying certain menu items, such as brie stuffed French toast and spinach goat cheese omelette, but others had more neutral or lukewarm opinions about their meals.","Mixed\n\nReviews praise the friendly and outgoing service, with several reviewers mentioning that their servers were extremely nice and attentive. However, some reviewers express frustration with noise levels, particularly after renovations added more hard surfaces, making it difficult to converse with table mates. Additionally, a few reviewers mention that the atmosphere can be disrupted by loud or obnoxious patrons, but this is not consistently mentioned as a major issue.","Reviews are mostly positive (7 out of 11) with some negative comments about noise levels and ambiance. Some reviewers appreciate the cozy atmosphere, while others find it too loud or obnoxious due to the presence of musicians/artists. One reviewer notes that the recent renovation has made the space feel more crowded and noisy.\n\nThe negative reviews highlight issues with noise levels, citing ""unbearable"" sound bouncing around due to hard surfaces. However, some positive reviewers mention enjoying the atmosphere and music, suggesting that it can be a pleasant experience for those who prefer a lively environment."
French Toast,"Mixed performance in the area of food, with some reviewers raving about high-quality dishes and others finding it merely ""good"" or ""unexceptional"". Positive comments highlight specific menu items such as pumpkin pancakes, sweet potato fries, eggs Benedict, and brie-stuffed French toast. However, a few reviews express disappointment with certain dishes, like the Crème Brulee Battered French Toast being just ""ok"", and one reviewer notes that the food is not exceptional enough to remember afterwards.","The service at French Toast is generally positive, with many reviewers praising the friendly and attentive staff. Several reviewers mention that the service was ""AWESOME"" (review 8) and that the server and owner were ""extremely nice"" (review 5). However, some reviewers have mixed opinions, with one reviewer stating that the staff seemed to be ""up their own ass"" due to constant praise from customers (review 9), while another reviewer found the atmosphere to be ""BLAH"" and the service to be merely ""fine"" (review 11).","Ambiance: Mixed\nFrench Toast's ambiance receives praise for its cozy environment, music setting, and friendly staff. However, some reviewers criticize the high noise levels, which can be overwhelming and make conversation difficult. The small space can also lead to long wait times and a crowded atmosphere that may not suit everyone's taste."
Spasso Italian Grill,"Mixed performance in the area of food, with some reviewers praising dishes such as pumpkin pancakes, sweet potato fries, and brie stuffed French toast, while others describe their meals as ""unexceptional"" or ""merely good"". Some reviewers also mention that certain items on the menu are particularly excellent, but do not specify which ones. Overall, it seems that Spasso Italian Grill has a solid foundation of quality food options, but may not stand out in terms of culinary innovation or exceptional dishes.","The service at Spasso Italian Grill is generally positive, with many reviewers praising the friendly and attentive staff. However, some reviewers have noted that the noise level can be a problem, particularly in the renovated space with its hard surfaces. Despite this, the overall consensus is that the service is excellent, with servers being described as ""extremely nice"" and able to accommodate requests, such as moving tables to a quieter area.","Mixed performan